<h1 align="center"><font color="yellow">PGVector: Crie, armazene e consulte Embeddings OpenAI no PostgreSQL usando pgvector</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

Vamos seguir os seguintes passos:

* docker pull ankane/pgvector

* docker-compose up -d

* Connect to the server 





In [56]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.pgvector import PGVector, DistanceStrategy
from langchain.docstore.document import Document

import os
import openai
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.getenv('OPENAI_API_KEY')


embeddings = OpenAIEmbeddings()

connection_string = PGVector.connection_string_from_db_params(
    driver=os.environ.get("DB_DRIVER", "psycopg"),
    host=os.getenv('DB_HOST'),
    port=os.getenv('DB_PORT'),
    database=os.getenv('DB_NAME'),
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD')
)


In [57]:
loader = TextLoader('./data/exemplo_DETRAN-DF.txt')
documents = loader.load()

In [58]:
documents

[Document(page_content='Fale com o Detran-DF\nCentral de Atendimento ao Cidadão\nO atendimento ao cidadão é feito pelo telefone 154, para quem está no DF, e 0800 644 0154, para quem está fora do DF.\nO Call Center funciona 24 horas por dia com atendimento eletrônico e, das 7h às 19h, em dias úteis, com atendimento humano.\nCerca de 40 atendentes trabalham no fornecimento de informações sobre os serviços prestados pelo Detran à população do DF. O usuário poderá pedir esclarecimentos sobre os diversos serviços do Órgão, tirar dúvidas, fazer reclamação, dar sugestão e até mesmo fazer elogios.\nCaso queira registrar uma reclamação, denúncia, sugestão, elogio ou informações de caráter geral sobre serviços, tais como horários de funcionamento, números de telefone, endereços, entre outros, ligue 162, número da Ouvidoria.\nSe quiser informar sobre irregularidades ou denúncias de urgência e emergência no trânsito, relacionadas à fiscalização, ligue 190 ou 193. Mas atenção!  Por ser um número de

In [60]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=250,
                                      chunk_overlap=60,
                                      separator="\n"
                                     )

docs = text_splitter.split_documents(documents)
print(len(documents))
print(len(docs))

Created a chunk of size 271, which is longer than the specified 250
Created a chunk of size 270, which is longer than the specified 250
Created a chunk of size 304, which is longer than the specified 250


1
11


In [61]:
docs

[Document(page_content='Fale com o Detran-DF\nCentral de Atendimento ao Cidadão\nO atendimento ao cidadão é feito pelo telefone 154, para quem está no DF, e 0800 644 0154, para quem está fora do DF.', metadata={'source': './data/exemplo_DETRAN-DF.txt'}),
 Document(page_content='O Call Center funciona 24 horas por dia com atendimento eletrônico e, das 7h às 19h, em dias úteis, com atendimento humano.', metadata={'source': './data/exemplo_DETRAN-DF.txt'}),
 Document(page_content='Cerca de 40 atendentes trabalham no fornecimento de informações sobre os serviços prestados pelo Detran à população do DF. O usuário poderá pedir esclarecimentos sobre os diversos serviços do Órgão, tirar dúvidas, fazer reclamação, dar sugestão e até mesmo fazer elogios.', metadata={'source': './data/exemplo_DETRAN-DF.txt'}),
 Document(page_content='Caso queira registrar uma reclamação, denúncia, sugestão, elogio ou informações de caráter geral sobre serviços, tais como horários de funcionamento, números de tele

In [62]:
from typing import List, Tuple

collection_name = 'DETRAN-DF'
# O Módulo PGVector tentará criar uma tabela com o nome da coleção (collection). 
# Portanto, certifique-se de que o nome da coleção seja exclusivo e que o usuário tenha permissão para criar uma tabela.
db = PGVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=collection_name,
    connection_string=connection_string,
)


In [70]:
query ="O que é a central de atendimento ao cidadão?" #"Quais são os requisitos necessários para a educação hospitalar?"
docs_with_score: List[Tuple[Document, float]] = db.similarity_search_with_score(query,
                                                                                k=4)


In [71]:
docs_with_score

[(Document(page_content='Fale com o Detran-DF\nCentral de Atendimento ao Cidadão\nO atendimento ao cidadão é feito pelo telefone 154, para quem está no DF, e 0800 644 0154, para quem está fora do DF.', metadata={'source': './data/exemplo_DETRAN-DF.txt'}),
  0.13554074441518482),
 (Document(page_content='O Call Center funciona 24 horas por dia com atendimento eletrônico e, das 7h às 19h, em dias úteis, com atendimento humano.', metadata={'source': './data/exemplo_DETRAN-DF.txt'}),
  0.14110601574583936),
 (Document(page_content='Cerca de 40 atendentes trabalham no fornecimento de informações sobre os serviços prestados pelo Detran à população do DF. O usuário poderá pedir esclarecimentos sobre os diversos serviços do Órgão, tirar dúvidas, fazer reclamação, dar sugestão e até mesmo fazer elogios.', metadata={'source': './data/exemplo_DETRAN-DF.txt'}),
  0.14203222179876396),
 (Document(page_content='Caso queira registrar uma reclamação, denúncia, sugestão, elogio ou informações de caráte

In [72]:
for doc, score in docs_with_score:
    print("-" * 100)
    print("Score: ", score)
    print("Conteúdo: ", doc.page_content)
    print("")
    print(doc.metadata)
    print("-" * 100)
    

----------------------------------------------------------------------------------------------------
Score:  0.13554074441518482
Conteúdo:  Fale com o Detran-DF
Central de Atendimento ao Cidadão
O atendimento ao cidadão é feito pelo telefone 154, para quem está no DF, e 0800 644 0154, para quem está fora do DF.

{'source': './data/exemplo_DETRAN-DF.txt'}
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Score:  0.14110601574583936
Conteúdo:  O Call Center funciona 24 horas por dia com atendimento eletrônico e, das 7h às 19h, em dias úteis, com atendimento humano.

{'source': './data/exemplo_DETRAN-DF.txt'}
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Score:  0.14203222179876396
Cont

In [73]:
store = PGVector(
    connection_string=connection_string, 
    embedding_function=embeddings, 
    collection_name= 'DETRAN-DF', 
    distance_strategy=DistanceStrategy.COSINE
)

retriever = store.as_retriever(search_kwargs={"k": 3})


In [74]:
retriever

VectorStoreRetriever(tags=['PGVector', 'OpenAIEmbeddings'], metadata=None, vectorstore=<langchain.vectorstores.pgvector.PGVector object at 0x7f7f5e5830d0>, search_type='similarity', search_kwargs={'k': 3})

In [76]:

retriever.get_relevant_documents(query="O que é a central de atendimento ao cidadão?")


[Document(page_content='Fale com o Detran-DF\nCentral de Atendimento ao Cidadão\nO atendimento ao cidadão é feito pelo telefone 154, para quem está no DF, e 0800 644 0154, para quem está fora do DF.', metadata={'source': './data/exemplo_DETRAN-DF.txt'}),
 Document(page_content='O Call Center funciona 24 horas por dia com atendimento eletrônico e, das 7h às 19h, em dias úteis, com atendimento humano.', metadata={'source': './data/exemplo_DETRAN-DF.txt'}),
 Document(page_content='Cerca de 40 atendentes trabalham no fornecimento de informações sobre os serviços prestados pelo Detran à população do DF. O usuário poderá pedir esclarecimentos sobre os diversos serviços do Órgão, tirar dúvidas, fazer reclamação, dar sugestão e até mesmo fazer elogios.', metadata={'source': './data/exemplo_DETRAN-DF.txt'})]